In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
from tqdm import tqdm
from datetime import timedelta
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

### 构建训练数据和验证数据
### 对训练数据，验证数据和测试数据做处理

In [2]:
### 北京市重点区域信息
cols = ['id', 'area_name', 'area_type', 'Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_info.csv', names=cols)

In [3]:
area_passenger_info.iloc[0:5]

,id,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,2,古塔公园,旅游景点;公园,116.535404,39.878676,116.535232,39.878140,575268.839844
2,3,中央戏剧学院,教育培训;高等院校,116.404667,39.935999,116.404148,39.936165,17024.636719
3,4,丰联广场,购物;购物中心,116.438556,39.923713,116.438117,39.924203,11695.019531
4,5,中华女子学院,教育培训;高等院校,116.435352,39.994935,116.434508,39.994636,83053.546875


In [4]:
### 重点区域人流量情况
cols = ['id', 'date/hour', 'index']
area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_index.csv', names=cols)
### 将时间戳拆成月，日和小时
### 去掉年份
area_passenger_index['date'] = area_passenger_index['date/hour'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))
area_passenger_index['date'] = pd.to_datetime(area_passenger_index['date'])
area_passenger_index['hour'] = area_passenger_index['date/hour'].apply(lambda x: int(str(x)[8:]))
### 删除原时间戳
area_passenger_index = area_passenger_index.drop(['date/hour'], axis=1)
area_passenger_index.shape

(717840, 4)

In [5]:
area_passenger_index.iloc[0:5]

,id,index,date,hour
0,1,1.8,2020-01-17,0
1,1,1.5,2020-01-17,1
2,1,1.3,2020-01-17,2
3,1,1.3,2020-01-17,3
4,1,1.7,2020-01-17,4


In [6]:
### 将北京市重点区域信息和重点区域人流量合并
area_passenger_index = area_passenger_index.merge(area_passenger_info,how = 'left',on = ['id'])
area_passenger_index.shape

(717840, 11)

In [7]:
area_passenger_index.iloc[0:3]

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,1.8,2020-01-17,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,1.5,2020-01-17,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,1.3,2020-01-17,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375


In [8]:
### 将网格联系强度添加至表格中
### 在每个时刻，计算某一区域与其他所有区域的网格联系强度之和（in），然后减去该区域去往其他区域的网格联系强度之和（out）
cols = ['hour', 'start_grid_x', 'start_grid_y', 'end_grid_x', 'end_grid_y', 'index']
grid_strength = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/datafountain_competition_od.txt', names=cols, encoding='gb2312', sep='\t')

In [9]:
del grid_strength

In [ ]:
### 计算每个区域在每个时刻的网格联系强度输入in
### 假设每天在同一时刻，网格联系强度相同
strenth_in = np.zeros((area_passenger_index.shape[0], 1)) - 1
strenth_in = pd.DataFrame(strenth_in)
strenth_in.columns = ['in']
strenth_out = np.zeros((area_passenger_index.shape[0], 1)) - 1
strenth_out = pd.DataFrame(strenth_out)
strenth_out.columns = ['out']

for index, row in tqdm(area_passenger_index.iterrows()):
    if (strenth_in.iloc[index]['in'] == -1) & (strenth_out.iloc[index]['out'] == -1):
        ### 计算 in
        ### 根据hour和end进行定位
        tem = grid_strength[(grid_strength['hour']==row['hour']) & (grid_strength['end_grid_x']==row['Grid_x']) & (grid_strength['end_grid_y']==row['Grid_y'])]['index']
        ### 求和
        tem_sum = tem.sum()
        if tem_sum != 0:
            print('tem_sum:', tem_sum)
        strenth_in[(area_passenger_index['id']==row['id']) & (area_passenger_index['hour']==row['hour'])] = tem_sum
        ### 计算 out
        tem = grid_strength[(grid_strength['hour']==row['hour']) & (grid_strength['start_grid_x']==row['Grid_x']) & (grid_strength['start_grid_y']==row['Grid_y'])]['index']
        ### 求和
        tem_sum = tem.sum()
        strenth_out[(area_passenger_index['id']==row['id']) & (area_passenger_index['hour']==row['hour'])] = tem_sum

241921it [18:16, 272.84it/s]

In [38]:
### 读取迁徙指数
cols = ['date', 'departure_province', 'departure_city', 'arrival__province', 'arrival_city', 'index']
migration_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/migration_index.csv', names=cols)
migration_index.shape

(21459, 6)

In [39]:
migration_index.iloc[0:5]

,date,departure_province,departure_city,arrival__province,arrival_city,index
0,20200117,陕西省,商洛市,北京市,北京市,0.00094
1,20200117,北京市,北京市,江西省,上饶市,0.01468
2,20200117,北京市,北京市,山东省,淄博市,0.03619
3,20200117,湖南省,怀化市,北京市,北京市,0.00288
4,20200117,山东省,淄博市,北京市,北京市,0.01698


In [40]:
### 对迁徙指数进行处理
### 从外地来北京，index增加
### 从北京去外地，index减少
### 计算出每天北京市整体的index
def process_migration_index(df):
    df['migration_index'] = 0
    out = pd.DataFrame()
    ### 将时间戳处理为时间格式
    df['date'] = df['date'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:]))
    df['date'] = pd.to_datetime(df['date'])
    start_time = df.iloc[0]['date']
    ### 当日期存在于表格中时
    for i in tqdm(range(df.shape[0])):
        if df[(df['date']==start_time)].index.tolist():
            ### 取出这一天的所有数据
            tem = df[df['date']==start_time]
            ### 指数初始化
            index_in = 0
            index_out = 0
            for index, row in tem.iterrows():
                ### 如果出发地是北京，则index_out增加
                if row['departure_city'] == '北京市':
                    index_out = index_out + 1
                ### 否则，index_in增加
                else:
                    index_in = index_in + 1
            ### 计算完一天所有的in,out之后，进行整合
            migration_index = index_in - index_out
#             print(migration_index)
            ### 将迁徙指数放入df
            df['migration_index'][df['date']==start_time] = migration_index
            ### 处理天数加一
            start_time = start_time + timedelta(days=1)
    ### 全部处理完成之后，按照日期去重复
    df = df.drop_duplicates(subset=['date'], keep='first').reset_index(drop=True)    
    return df

In [41]:
migration_index = process_migration_index(df=migration_index)

100%|██████████| 21459/21459 [00:37<00:00, 570.50it/s]


In [42]:
migration_index.iloc[0:5]

,date,departure_province,departure_city,arrival__province,arrival_city,index,migration_index
0,2020-01-17,陕西省,商洛市,北京市,北京市,0.00094,-5
1,2020-01-18,甘肃省,酒泉市,北京市,北京市,0.00188,-4
2,2020-01-19,北京市,北京市,海南省,白沙黎族自治县,0.00058,-8
3,2020-01-20,新疆维吾尔自治区,博尔塔拉蒙古自治州,北京市,北京市,0.00016,-11
4,2020-01-21,北京市,北京市,浙江省,温州市,0.04604,-11


In [44]:
### 将日期和迁移指数保存
migration_index = migration_index[['date', 'migration_index']]
migration_index.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/migration_index.csv', index=None)

In [45]:
migration_index.iloc[-10:]

,date,migration_index
20,2020-02-06,21
21,2020-02-07,12
22,2020-02-08,26
23,2020-02-09,17
24,2020-02-10,17
25,2020-02-11,7
26,2020-02-12,14
27,2020-02-13,22
28,2020-02-14,29
29,2020-02-15,4


In [10]:
area_passenger_index.iloc[0:5]

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,1.8,2020-01-17,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,1.5,2020-01-17,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,1.3,2020-01-17,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
3,1,1.3,2020-01-17,3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
4,1,1.7,2020-01-17,4,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375


In [10]:
### 给数据打上label
### 将每个时刻往后推9天，为了使线上提交测试集的最后一天有数据可用
### 取出对应的index作为预测label
area_passenger_index['label'] = 0
labels = np.zeros((area_passenger_index.shape[0], 1))
labels = pd.DataFrame(labels)
labels.columns = ['label']
max_date = area_passenger_index['date'].iloc[-1]
for index, row in tqdm(area_passenger_index.iterrows()):
    date_forward = row['date']+timedelta(days=9)
    if date_forward <= max_date:
        ### 先找出日期符合的所有数据
        tem = area_passenger_index[area_passenger_index['date']==date_forward]
        ### 再通过id和hour找出唯一符合的数据
        tem = tem[(tem['id']==row['id']) & (tem['hour']==row['hour'])]
        labels.iloc[index]['label'] = tem['index']
    else:
        labels.iloc[index]['label'] = -999

717840it [49:38, 240.97it/s]


In [11]:
area_passenger_index['label'] = labels

In [12]:
area_passenger_index.iloc[0:15]

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area,label
0,1,1.8,2020-01-17,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2.0
1,1,1.5,2020-01-17,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.5
2,1,1.3,2020-01-17,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.8
3,1,1.3,2020-01-17,3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.3
4,1,1.7,2020-01-17,4,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.4
5,1,1.2,2020-01-17,5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.6
6,1,2.9,2020-01-17,6,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2.3
7,1,4.7,2020-01-17,7,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,3.3
8,1,6.1,2020-01-17,8,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,3.9
9,1,5.2,2020-01-17,9,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,5.7


In [ ]:
### 根据线上测试数，从训练集中找出需要用到的数据
sub = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
sub = sub.merge(area_passenger_info, how='left', on=['id'])
sub['date'] = sub['date/hour'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))
sub['date'] = pd.to_datetime(sub['date'])
sub['hour'] = sub['date/hour'].apply(lambda x: int(str(x)[8:]))
sub = sub.drop(['date/hour'], axis=1)
input_for_sub = pd.DataFrame()
### 将sub的日期往前推9天，找出需要的数据
for index, row in tqdm(sub.iterrows()):
    date_forward = row['date']-timedelta(days=9)
    ### 先找出日期符合的所有数据
    tem = area_passenger_index[(area_passenger_index['date']==date_forward) & (area_passenger_index['id']==row['id']) & (area_passenger_index['hour']==row['hour'])]
    input_for_sub = pd.concat([input_for_sub, tem], axis=0)

214505it [1:20:10, 26.19it/s]

In [28]:
### 重置索引
input_for_sub = input_for_sub.reset_index(drop=True)

In [29]:
input_for_sub

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area,label
0,1,0.6,2020-02-07,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2.1
1,1,0.7,2020-02-07,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.1
2,1,1.6,2020-02-07,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,0.9
3,1,0.5,2020-02-07,3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.3
4,1,0.9,2020-02-07,4,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.3
5,1,0.8,2020-02-07,5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.6
6,1,1.3,2020-02-07,6,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2.2
7,1,3.0,2020-02-07,7,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,3.7
8,1,3.1,2020-02-07,8,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,4.0
9,1,3.3,2020-02-07,9,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,3.6


In [86]:
### 打乱顺序
area_passenger_index = shuffle(area_passenger_index).reset_index(drop=True)
### 提出特征和label
label = area_passenger_index['index']
features  = area_passenger_index.drop(['index'], axis=1)

In [87]:
features.iloc[0:5]

,month,day,hour,Center_x,Center_y,Grid_x,Grid_y,area
0,1,20,15,0.704571,0.278811,0.703781,0.279100,0.003549
1,2,5,22,0.508123,0.458757,0.508520,0.458611,0.000163
2,1,28,15,0.900823,0.903021,0.901164,0.902406,0.005632
3,1,31,17,0.927608,0.446886,0.927536,0.445391,0.011926
4,2,1,22,0.415220,0.964700,0.414568,0.966379,0.009625


In [92]:
### 定义模型参数
params = {'num_leaves': 38,
          'min_data_in_leaf': 50,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.02,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 0.9,
          "bagging_freq": 1,
          "bagging_fraction": 0.7,
          "bagging_seed": 11,
          "lambda_l1": 0.1,
          "verbosity": -1,
          "nthread": 4,
          'metric': 'mae',
          "random_state": 2019,
#           'device': 'gpu'
          }

In [93]:
def r_mean_absolute_percentage_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [97]:
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

In [98]:
### K 折
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    trn_data = lgb.Dataset(features.iloc[trn_idx], label=label.iloc[trn_idx])
    val_data = lgb.Dataset(features.iloc[val_idx], label=label.iloc[val_idx])
    num_round = 1000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=200,
                    early_stopping_rounds=200)
    pred = clf.predict(features.iloc[val_idx], num_iteration=clf.best_iteration)
    print('RMSE:', r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred))
    print('Score:', 1/(1+r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred)))

fold 1
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 16.1376	valid_1's l1: 16.2097
[400]	training's l1: 12.8509	valid_1's l1: 12.9447
[600]	training's l1: 11.1913	valid_1's l1: 11.2909
[800]	training's l1: 10.2075	valid_1's l1: 10.3177
[1000]	training's l1: 9.38185	valid_1's l1: 9.49428
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 9.38185	valid_1's l1: 9.49428
RMSE: 22.00117211298944
Score: 0.04347604526793965
fold 2
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 15.9958	valid_1's l1: 16.1521
[400]	training's l1: 12.6786	valid_1's l1: 12.8073
[600]	training's l1: 11.1071	valid_1's l1: 11.2312
[800]	training's l1: 10.1123	valid_1's l1: 10.2369
[1000]	training's l1: 9.30492	valid_1's l1: 9.43417
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 9.30492	valid_1's l1: 9.43417
RMSE: 21.52018355788359
Score: 0.04440461142022673
fold 3
Training until validation scores don't im

In [138]:
### submit
sub = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
submit = sub.copy(deep=True)
### merge
sub = sub.merge(area_passenger_info, how='left', on=['id'])
### 对时间戳进行处理
sub['month'] = sub['date/hour'].apply(lambda x: int(''.join(str(x)[4:6])))
sub['day'] = sub['date/hour'].apply(lambda x: int(''.join(str(x)[6:8])))
sub['hour'] = sub['date/hour'].apply(lambda x: int(str(x)[8:]))
### 删除原时间戳
sub = sub.drop(['date/hour'], axis=1)
### 删除id, area_name, area_type, index
sub = sub.drop(['id', 'area_name', 'area_type', 'index'], axis=1)
### 归一化
sub[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(sub[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
sub.iloc[0:5]
### 预测
pred = clf.predict(sub)
### 将预测结果作为index放入submit
submit['index'] = pred
### 保存
submit.to_csv('/home/poac/code/Block_predict/submit/block_predict_result.csv', index=None, header=None)